In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf

In [ ]:
# datasets
ACNE_PATH = "Dataset/Acne"
PSORIASIS_PATH = "Dataset/Psoriasis"
ECZEMA_PATH = "Dataset/Eczema"

In [ ]:
# Création d'un dataframe pour organiser les images et leurs labels
def create_dataframe(acne_path, psoriasis_path, eczema_path):
    data = []
    
    # Ajout des images d'acné
    for img_name in os.listdir(acne_path):
        if img_name.endswith(('.jpg', '.jpeg', '.png')):
            data.append({
                'image_path': os.path.join(acne_path, img_name),
                'label': 'acne'
            })
    
    # Ajout des images de psoriasis
    for img_name in os.listdir(psoriasis_path):
        if img_name.endswith(('.jpg', '.jpeg', '.png')):
            data.append({
                'image_path': os.path.join(psoriasis_path, img_name),
                'label': 'psoriasis'
            })
    
    # Ajout des images d'eczéma
    for img_name in os.listdir(eczema_path):
        if img_name.endswith(('.jpg', '.jpeg', '.png')):
            data.append({
                'image_path': os.path.join(eczema_path, img_name),
                'label': 'eczema'
            })
    
    return pd.DataFrame(data)

# Création du dataframe
df = create_dataframe(ACNE_PATH, PSORIASIS_PATH, ECZEMA_PATH)

# Vérification de la distribution des classes
print(df['label'].value_counts())

# Division en train, validation et test sets
train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

# Paramètres pour le prétraitement des images
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Augmentation des données pour l'ensemble d'entraînement
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [ ]:
# Pas d'augmentation pour la validation et le test
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Générateurs de données
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='image_path',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Construction du modèle

In [ ]:
def build_model(num_classes):
    # Chargement du modèle pré-entraîné EfficientNetB0
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
    )
    
    # Congélation des couches du modèle de base
    base_model.trainable = False
    
    # Ajout de nouvelles couches pour la classification
    inputs = tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    
    return model

# Construction du modèle
num_classes = len(train_generator.class_indices)
model = build_model(num_classes)

# Compilation du modèle
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
# Affichage de l'architecture du modèle
model.summary()

# Entraînement du modèle

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

# Entraînement
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=30,
    validation_data=val_generator,
    validation_steps=val_generator.n // val_generator.batch_size,
    callbacks=[early_stopping, model_checkpoint]
)

In [ ]:
# Sauvegarde du modèle final
model.save('dermatology_model.h5')

# Évaluation du modèle

In [ ]:
# Fonction pour afficher l'historique d'entraînement
def plot_training_history(history):
    plt.figure(figsize=(12, 4))
    
    # Courbe de précision
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy over epochs')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    
    # Courbe de loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss over epochs')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Affichage des courbes d'apprentissage
plot_training_history(history)

# Évaluation sur l'ensemble de test
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

In [ ]:
# Matrice de confusion
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Prédictions sur l'ensemble de test
test_generator.reset()
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes

# Noms des classes
class_names = list(test_generator.class_indices.keys())

# Matrice de confusion
conf_mat = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# Rapport de classification
print(classification_report(y_true, y_pred_classes, target_names=class_names))

# Fonction de prédiction sur de nouvelles images

In [ ]:
from tensorflow.keras.preprocessing import image

def predict_skin_disease(image_path, model, class_names):
    """
    Prédit la maladie dermatologique à partir d'une image
    
    Args:
        image_path: chemin vers l'image à analyser
        model: modèle entraîné
        class_names: liste des noms des classes
        
    Returns:
        Dictionnaire avec les probabilités pour chaque classe
    """
    # Chargement et prétraitement de l'image
    img = image.load_img(image_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    
    # Prédiction
    predictions = model.predict(img_array)
    
    # Création du résultat
    result = {class_names[i]: float(predictions[0][i]) for i in range(len(class_names))}
    
    return result

In [ ]:
# Exemple d'utilisation
new_image_path = "chemin/vers/nouvelle_image.jpg"
predictions = predict_skin_disease(new_image_path, model, class_names)
print("Résultats de la prédiction:")
for disease, prob in predictions.items():
    print(f"{disease}: {prob:.2%}")